<a href="https://colab.research.google.com/github/hellwalker3/naive-txt2img-api-of-sd/blob/main/naive_text2img_api_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install

# webui
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# models
!wget https://huggingface.co/dreamlike-art/dreamlike-anime-1.0/resolve/main/dreamlike-anime-1.0.safetensors -O /content/stable-diffusion-webui/models/Stable-diffusion/dreamlike-anime-1.0.safetensors 
!wget https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors -O /content/stable-diffusion-webui/models/embeddings/EasyNegative.safetensors

# extensions
!cd /content/stable-diffusion-webui/extensions && git clone ttps://github.com/yfszzx/stable-diffusion-webui-images-browser.git
!cd /content/stable-diffusion-webui/extensions && git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger.git
!cd /content/stable-diffusion-webui/extensions && git clone https://github.com/Mikubill/sd-webui-controlnet.git
!cd /content/stable-diffusion-webui/extensions && git clone https://github.com/kohya-ss/sd-webui-additional-networks.git
#!cd /content/stable-diffusion-webui/extensions && git clone https://github.com/opparco/stable-diffusion-webui-two-shot.git

# controlnet and T2I-Adapter
!wget https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/t2iadapter_keypose_sd14v1.pth
!wget https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/t2iadapter_color_sd14v1.pth
#!wget https://huggingface.co/kohya-ss/ControlNet-diff-modules/resolve/main/diff_control_sd15_seg_fp16.safetensors -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/diff_control_sd15_seg_fp16.safetensors
!wget https://huggingface.co/kohya-ss/ControlNet-diff-modules/resolve/main/diff_control_sd15_canny_fp16.safetensors -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/diff_control_sd15_canny_fp16.safetensors
#!wget https://huggingface.co/kohya-ss/ControlNet-diff-modules/resolve/main/diff_control_sd15_normal_fp16.safetensors -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/diff_control_sd15_normal_fp16.safetensors
#!wget https://huggingface.co/kohya-ss/ControlNet-diff-modules/resolve/main/diff_control_sd15_depth_fp16.safetensors -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/diff_control_sd15_depth_fp16.safetensors

# modules
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
#@title Using an older version of controlnet due to difficulty with the latest version and passing special instances as arguments to the txt2img function
!cd /content/stable-diffusion-webui/extensions/sd-webui-controlnet && git checkout b6fc297

In [ ]:
#@title Copy LoRA file (if you have)
from glob import glob
lora_folder_path = "/content/drive/MyDrive/dreambooth_models"
for file in glob(f'{lora_folder_path}/*'):
      !cp {file} /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora

In [ ]:
#@title Add txt2img2 API (Modifications required for extensions needing non-controlnet images, as they're not supported)
#I believe it's a bad practice, but I am overwriting the file

base_path ='/content'

add_api = """
import os
from contextlib import redirect_stdout
class Api(Api):
    def __init__(self, app: FastAPI, queue_lock: Lock):
        super().__init__(app, queue_lock)
        self.add_api_route("/sdapi/v1/txt2img2", self.text2imgapi2, methods=["POST"], response_model=dict)

    def text2imgapi2(self, txt2imgreq: dict):
        with redirect_stdout(open(os.devnull, 'w')):
            from modules.generation_parameters_copypaste import create_override_settings_dict
            override_settings = create_override_settings_dict(txt2imgreq['override_settings_texts'])
            p = StableDiffusionProcessingTxt2Img(
                sd_model=shared.sd_model,
                do_not_save_samples=True,
                do_not_save_grid=True,
                prompt=txt2imgreq['prompt'],
                negative_prompt=txt2imgreq['negative_prompt'],
                seed=txt2imgreq['seed'],
                subseed=txt2imgreq['subseed'],
                subseed_strength=txt2imgreq['subseed_strength'],
                seed_resize_from_h=txt2imgreq['seed_resize_from_h'],
                seed_resize_from_w=txt2imgreq['seed_resize_from_w'],
                seed_enable_extras=txt2imgreq['seed_enable_extras'],
                sampler_name=sd_samplers.samplers[txt2imgreq['sampler_index']].name,
                batch_size=txt2imgreq['batch_size'],
                n_iter=txt2imgreq['n_iter'],
                steps=txt2imgreq['steps'],
                cfg_scale=txt2imgreq['cfg_scale'],
                width=txt2imgreq['width'],
                height=txt2imgreq['height'],
                restore_faces=txt2imgreq['restore_faces'],
                tiling=txt2imgreq['tiling'],
                enable_hr=txt2imgreq['enable_hr'],
                denoising_strength=txt2imgreq['denoising_strength'] if txt2imgreq['enable_hr'] else None,
                hr_scale=txt2imgreq['hr_scale'],
                hr_upscaler=txt2imgreq['hr_upscaler'],
                hr_second_pass_steps=txt2imgreq['hr_second_pass_steps'],
                hr_resize_x=txt2imgreq['hr_resize_x'],
                hr_resize_y=txt2imgreq['hr_resize_y'],
                override_settings=override_settings,
            )
            p.scripts = scripts.scripts_txt2img
            import numpy as np
            def decode_dict(dct):
                dct['image'] = np.array(decode_base64_to_image(dct['image']))
                dct['mask'] = np.tile([0, 0, 0, 225], dct['image'].shape[:2]).reshape(*dct['image'].shape[:2], 4).astype(np.uint8)

            def decode_tuple(tpl):
                lst = list(tpl)
                for i in range(len(lst)):
                    if isinstance(lst[i], dict):
                        decode_dict(lst[i])
                return tuple(lst)

            args = decode_tuple(txt2imgreq['args'])
            p.script_args = args

        
            with self.queue_lock:
                shared.state.begin()
                processed = scripts.scripts_txt2img.run(p, *args)
                if processed is None:
                    processed = process_images(p)
                p.close()
                shared.total_tqdm.clear()
                shared.state.end()

        b64images = list(map(encode_pil_to_base64, processed.images))

        return {'images':b64images}
        #, 'info':processed.js()}
"""

file_path = f'{base_path}/stable-diffusion-webui/modules/api/api.py'
program_to_append = add_api
with open(file_path, 'a') as f:
    f.write('\n' + program_to_append)

In [ ]:
#@title Add a process to display the arguments passed during txt2img execution in the web UI (for use when running the API).

target_file = f'{base_path}/stable-diffusion-webui/modules/txt2img.py'

with open(target_file, 'r') as f:
    contents = f.read()


old_str = """
def txt2img(id_task: str, prompt: str, negative_prompt: str, prompt_styles, steps: int, sampler_index: int, restore_faces: bool, tiling: bool, n_iter: int, batch_size: int, cfg_scale: float, seed: int, subseed: int, subseed_strength: float, seed_resize_from_h: int, seed_resize_from_w: int, seed_enable_extras: bool, height: int, width: int, enable_hr: bool, denoising_strength: float, hr_scale: float, hr_upscaler: str, hr_second_pass_steps: int, hr_resize_x: int, hr_resize_y: int, override_settings_texts, *args):
"""
new_str = """
def txt2img(id_task: str, prompt: str, negative_prompt: str, prompt_styles, steps: int, sampler_index: int, restore_faces: bool, tiling: bool, n_iter: int, batch_size: int, cfg_scale: float, seed: int, subseed: int, subseed_strength: float, seed_resize_from_h: int, seed_resize_from_w: int, seed_enable_extras: bool, height: int, width: int, enable_hr: bool, denoising_strength: float, hr_scale: float, hr_upscaler: str, hr_second_pass_steps: int, hr_resize_x: int, hr_resize_y: int, override_settings_texts, *args):
    def print_as_dict(id_task, prompt, negative_prompt, prompt_styles, steps, sampler_index, restore_faces, tiling, n_iter, batch_size, cfg_scale, seed, subseed, subseed_strength, seed_resize_from_h, seed_resize_from_w, seed_enable_extras, height, width, enable_hr, denoising_strength, hr_scale, hr_upscaler, hr_second_pass_steps, hr_resize_x, hr_resize_y, override_settings_texts, *args):
        variables = {
            'id_task': id_task,
            'prompt': prompt,
            'negative_prompt': negative_prompt,
            'prompt_styles': prompt_styles,
            'steps': steps,
            'sampler_index': sampler_index,
            'restore_faces': restore_faces,
            'tiling': tiling,
            'n_iter': n_iter,
            'batch_size': batch_size,
            'cfg_scale': cfg_scale,
            'seed': seed,
            'subseed': subseed,
            'subseed_strength': subseed_strength,
            'seed_resize_from_h': seed_resize_from_h,
            'seed_resize_from_w': seed_resize_from_w,
            'seed_enable_extras': seed_enable_extras,
            'height': height,
            'width': width,
            'enable_hr': enable_hr,
            'denoising_strength': denoising_strength,
            'hr_scale': hr_scale,
            'hr_upscaler': hr_upscaler,
            'hr_second_pass_steps': hr_second_pass_steps,
            'hr_resize_x': hr_resize_x,
            'hr_resize_y': hr_resize_y,
            'override_settings_texts': override_settings_texts,
            'args': args
        }
        print(variables)
    
    print_as_dict(id_task, prompt, negative_prompt, prompt_styles, steps, sampler_index, restore_faces, tiling, n_iter, batch_size, cfg_scale, seed, subseed, subseed_strength, seed_resize_from_h, seed_resize_from_w, seed_enable_extras, height, width, enable_hr, denoising_strength, hr_scale, hr_upscaler, hr_second_pass_steps, hr_resize_x, hr_resize_y, override_settings_texts, *args)
"""
contents = contents.replace(old_str, new_str)

with open(target_file, 'w') as f:
    f.write(contents)


In [ ]:
#@title Launching webui and API server (for example, when using multiple controlnet instances, update settings and restart)
!COMMANDLINE_ARGS="--api --enable-insecure-extension-access --share --disable-safe-unpickle --xformers" REQS_FILE="requirements.txt" python launch.py

In [ ]:
#@title  First, perform a sample run in the webui to check the arguments passed to txt2img (arguments will be displayed in dictionary format starting with 'id_task'), then execute the API based on those arguments (naive_text2img_api_client.ipynb)